In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la

In [2]:
with open('screen_info.txt','rb') as fl:
    t = pickle.load(fl)
fnames = t[0]
totf = t[1]
binf = t[2]
runfile = 3
fname = fnames[runfile]
bf = binf[runfile]

In [3]:
print(fname)

AID688


In [4]:
path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints = []
c_fingerprints = []
labels = []
with open(path+fname+'red_train.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:bf])
        c_fingerprints.append(row[bf:-1])
        labels.append(row[-1])

### Using 10 fold cross validation 

In [5]:
p_fingerprints = np.asarray(p_fingerprints)[1:]
p_fingerprints = p_fingerprints.astype(int)

c_fingerprints = np.asarray(c_fingerprints)[1:]
c_fingerprints = c_fingerprints.astype(float)

#Normalise the features
c_fingerprints = (c_fingerprints - np.mean(c_fingerprints,axis=0))/np.std(c_fingerprints,axis=0)

#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = p_fingerprints.shape
ip_dim2 = c_fingerprints.shape[1]
labels = labels[1:]
print(no_examples,ip_dim)
print(ip_dim2)
p_fingerprints[(p_fingerprints==0)] = -1


(21751, 121)
32


In [6]:
labels2 = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2[i] = 1
    else:
        labels2[i] = 0
labels2 = labels2.astype(int)
total_pos = np.sum(labels2)
print("tot_positive",total_pos)
fp_allowed = int((no_examples-total_pos)*0.05)
print("fp allowed",fp_allowed)

('tot_positive', 198)
('fp allowed', 1077)


#####  Rnadomly permute the data

In [7]:
ind = np.arange(no_examples)
np.random.shuffle(ind)
p_fingerprints = p_fingerprints[ind]
c_fingerprints = c_fingerprints[ind]
labels2 = labels2[ind]

###  Find angle between mean vectors

In [8]:
# p_active = np.float32(np.sum(c_fingerprints[labels2[:,0]==1],axis=0))/np.sum(labels2)
# p_inactive = np.float32(np.sum(c_fingerprints[labels2[:,0]==0],axis=0))/(no_examples-np.sum(labels2))

# print p_active.shape
# ta = np.dot(p_active,p_inactive)
# print ta
# t1 = la.norm(p_active)
# t2 = la.norm(p_inactive)
# print t1,t2,t1*t2
# theta = np.float32(np.dot(p_active,p_inactive))/(la.norm(p_active)*la.norm(p_inactive))
# print(theta)

In [9]:
# print(p_active,p_inactive)
# print(p_active+p_inactive)

In [10]:
# def get_train_batch(batch_size,binary=True):
#     samples = np.random.randint(low=0,high=no_examples,size=(batch_size,1))
#     if binary == True:
#         train_batch = p_fingerprints[samples].reshape(batch_size,ip_dim)
#         train_batch = train_batch.astype(int)
#     else:
#         train_batch = c_fingerprints[samples].reshape(batch_size,ip_dim2)
#         train_batch = train_batch.astype(float)
    
#     train_batch = torch.cuda.FloatTensor(train_batch)
#     train_batch = Variable(train_batch,requires_grad=False).cuda()
#     #print(labels2[samples])
#     target = Variable(torch.cuda.LongTensor(labels2[samples]),requires_grad=False)
#     target = target.view(batch_size,)
#     #print(target.type)
#     return train_batch,target

In [11]:
no_active_ele = (sum(labels2))
print(no_active_ele)
batch_size = 400

[198]


In [12]:
def get_train_batch(batch_size,validation_iter=0,binary=True):
    
    if validation_iter == 0: #no validation
        curr_data_size = no_examples
        labels_train = labels2
    else:
        curr_data_size = int(no_examples*0.8)
        interval_size = int(no_examples*0.2)
        
        if(val_iter==1):
            s_ind1 = int((validation_iter)*interval_size)
            end_ind1 = int((validation_iter+1)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        else:
            s_ind1 = 0
            end_ind1 = int((validation_iter)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        
        #print("train_ind ",s_ind1,end_ind1,s_ind2,end_ind2)
        indices = range(s_ind1,end_ind1) + range(s_ind2,end_ind2)
        c_train_data = c_fingerprints[indices]
        p_train_data = p_fingerprints[indices]
        labels_train = labels2[indices]
                               
    samples = np.random.randint(low=0,high=curr_data_size,size=(batch_size,1))
    if binary == True:
        train_batch = p_train_data[samples].reshape(batch_size,ip_dim)
        train_batch = train_batch.astype(int)
    else:
        train_batch = c_train_data[samples].reshape(batch_size,ip_dim2)
        train_batch = train_batch.astype(float)
    
    train_batch = torch.cuda.FloatTensor(train_batch)
    train_batch = Variable(train_batch,requires_grad=False).cuda()
    target = Variable(torch.cuda.LongTensor(labels_train[samples]),requires_grad=False)
    target = target.view(batch_size,)
    return train_batch,target

In [13]:
def get_val_data(validation_iter,binary = True):
    interval_size = int(no_examples)*0.2
    s_ind = int((validation_iter-1)*interval_size)
    e_ind = int((validation_iter) * interval_size)
    if(binary==True):
        train_data = p_fingerprints[s_ind:e_ind]
    else:
        train_data = c_fingerprints[s_ind:e_ind]
    labels_val = labels2[s_ind:e_ind]   
    #print("val ind ",s_ind,e_ind)
    #print train_data.shape, labels_val.shape
    return Variable(torch.cuda.FloatTensor(train_data)),labels_val  
        

In [ ]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim2,1000)
        self.l2 = nn.Linear(1000,500)
        self.l3 = nn.Linear(500,500)
        self.l4 = nn.Linear(500,50)
        self.l5 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        x = F.leaky_relu(self.l4(x))
        x = (self.l5(x))
        
        return x

In [ ]:
min_fn = 15
max_fp = 200
maxtmpdiff = -10
cm_list = []
get_model = 1
for val_iter in range(1,6):
    print("val iter: ",val_iter)
    
   
    #weights_array = [5]
    weights_array = np.linspace(100,120,10)
    for i,w in enumerate(weights_array): 
        mymlp = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-4)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(2000):
            train_batch,target = get_train_batch(batch_size,binary = False,validation_iter = val_iter)
            model_op = mymlp(train_batch)
            #print(model_op.type)
            #print(target.type)
            loss = criterion(model_op,target)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    #     if(ep%30==29):
    #         print(loss.data[0])

        ## After training check on cross validation data
        val_data,labels_val = get_val_data(val_iter,binary = False)
        train_op = mymlp(val_data)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
        #tmp_labels = tmp_labels.data.cpu().numpy()
        #print(sum(tmp_labels))
        cf = metrics.confusion_matrix(labels_val,pred_labels).ravel()
        #print(val_iter,w)
        print('tn, fp, fn, tp: ',cf)
        [tn,fp,fn,tp] = cf
        wcf = [val_iter] + [w] + [cf]
        tmpdiff = tp-fn
        if(tmpdiff > maxtmpdiff):
            cm_list.append(wcf)
            if(tmpdiff<maxtmpdiff):
                if(fp < max_fp):
                    max_fp = fp
                    maxtmpdiff = tmpdiff
                    model_path = os.getcwd() + '/cnt_selu_' + fname + '_' + str(get_model)
                    torch.save(mymlp.state_dict(),model_path)
                    print("saving model on val: ",val_iter," and weight: ",w)
                    if(get_model<4):
                        get_model + get_model + 1
                    else:
                        break

        


('val iter: ', 1)
('tn, fp, fn, tp: ', array([4170,  137,   42,    1]))
('tn, fp, fn, tp: ', array([4150,  157,   40,    3]))
('tn, fp, fn, tp: ', array([4278,   29,   43,    0]))
('tn, fp, fn, tp: ', array([4035,  272,   40,    3]))
('tn, fp, fn, tp: ', array([3992,  315,   40,    3]))
('tn, fp, fn, tp: ', array([3997,  310,   41,    2]))
('tn, fp, fn, tp: ', array([4020,  287,   41,    2]))
('tn, fp, fn, tp: ', array([3710,  597,   37,    6]))
('tn, fp, fn, tp: ', array([3542,  765,   34,    9]))
('tn, fp, fn, tp: ', array([3487,  820,   35,    8]))
('val iter: ', 2)
('tn, fp, fn, tp: ', array([3823,  479,   39,    9]))
('tn, fp, fn, tp: ', array([3895,  407,   37,   11]))
('tn, fp, fn, tp: ', array([3835,  467,   34,   14]))
('tn, fp, fn, tp: ', array([3473,  829,   31,   17]))
('tn, fp, fn, tp: ', array([3671,  631,   34,   14]))
('tn, fp, fn, tp: ', array([3521,  781,   30,   18]))
('tn, fp, fn, tp: ', array([3055, 1247,   24,   24]))
('tn, fp, fn, tp: ', array([2971, 1331,   23, 

In [ ]:
cm_list

In [ ]:
# #weights_array = np.linspace(3,25,22)
# tmp_w = [weights_array[0]]
# cont_cm_lsit = []
# for i,w in enumerate(tmp_w):
    
    
#     c_mymlp = c_mlp().cuda()
#     optimizer2 = torch.optim.Adam(c_mymlp.parameters(),lr=1e-4)
    
#     weights = torch.cuda.FloatTensor([1,w])
#     criterion = nn.CrossEntropyLoss(weight=weights)
    
#     for ep in range(2000):
#         train_batch,target = get_train_batch(batch_size,binary=False)
#         #train_batch,target = sample_X(batch_size,labels='True')
#         #target = Variable(torch.cuda.LongTensor(target),requires_grad = False)
#         model_op = c_mymlp(train_batch)
#         #print(model_op.type)
#         #print(target.type)
#         loss = criterion(model_op,target)
#         loss.backward()
#         optimizer2.step()
#         optimizer2.zero_grad()
    
#         if(ep%100==99):
#             print(loss.data[0])

#     train_op = c_mymlp(Variable(torch.cuda.FloatTensor(c_fingerprints)))
#     train_op = train_op.cpu().data.numpy()
#     pred_labels = np.argmax(train_op,axis=1)
#     #tmp_labels = tmp_labels.data.cpu().numpy()
#     #print(sum(tmp_labels))
#     cont_cf = metrics.confusion_matrix(labels2,pred_labels).ravel()
#     print('tn, fp, fn, tp: ',cont_cf)
#     cont_cm_lsit.append(cont_cf)
    
#     #model_path = os.getcwd() + '/model' + str(i)
#     #torch.save(c_mymlp.state_dict(),model_path)
    

### Checking on Test data

In [ ]:
path = os.getcwd()+'/bioassay-datasets/'
test_bin = []
test_con = []
labels = []
with open(path+'AID362red_test.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        test_bin.append(row[:112])
        test_con.append(row[112:-1])
        labels.append(row[-1])

In [ ]:
test_bin = np.asarray(test_bin)[1:]
test_bin = test_bin.astype(int)

test_con = np.asarray(test_con)[1:]
test_con = test_con.astype(float)

#Normalise the features
test_con = (test_con - np.mean(test_con,axis=0))/np.std(test_con,axis=0)

#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = test_bin.shape
ip_dim2 = test_con.shape[1]
labels = labels[1:]
print(no_examples,ip_dim)
print(ip_dim2)
test_bin[(test_bin==0)] = -1


In [ ]:
best_c_mlp = c_mlp().cuda()
param_path = os.getcwd() + '/cont_model_with_val'
best_c_mlp.load_state_dict(torch.load(param_path))

In [ ]:
test_labels2 = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        test_labels2[i] = 1
    else:
        test_labels2[i] = 0
test_labels2 = test_labels2.astype(int)
total_pos = np.sum(test_labels2)
print("tot_positive",total_pos)

In [ ]:
test_op = best_c_mlp(Variable(torch.cuda.FloatTensor(test_con)))
test_op = test_op.cpu().data.numpy()
test_pred = np.argmax(test_op,axis=1)
    #tmp_labels = tmp_labels.data.cpu().numpy()
    #print(sum(tmp_labels))
t_cont_cf = metrics.confusion_matrix(test_labels2,test_pred).ravel()
print('tn, fp, fn, tp: ',t_cont_cf)